<a href="https://colab.research.google.com/github/AmritaNeogi/PhenoCam-Image-Analysis-Using-CNN/blob/main/Phenophase_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install torchvision

In [ ]:
!pip install tensorflow


In [ ]:
!pip install keras

In [3]:
import csv
import cv2
import os
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Non-Hazy Images
We already possess the code for handling 32 misty files. We will analyze our dataset to identify all the files that are free from haze or fog, and subsequently, we will store the images in a CSV file.

In [ ]:
not_foggy_file = "/content/drive/MyDrive/Capstone/data_out/not_foggy/NEON.D01.HARV.DP1.00033/2017.csv"

with open(not_foggy_file, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            line_count += 1
        img_name = row["file"]
        print(img_name)
        begin_cv_img = cv2.imread(f"/content/drive/MyDrive/Capstone/data_raw/phenocamdata/{img_name}")
        begin_rgb = cv2.cvtColor(begin_cv_img, cv2.COLOR_BGR2RGB)
        plt.imshow(begin_rgb)
        plt.show()

    print(f'Processed {line_count} lines.')

We already possess the code for handling 32 misty files. We will analyze our dataset to identify all the files that are free from haze or fog, and here we are saving the images in a folder.

In [ ]:
# Create the output folder if it doesn't exist
# Define the output folder
output_folder = "/content/drive/MyDrive/Capstone/data_working/images_2017"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

not_foggy_file = "/content/drive/MyDrive/Capstone/data_out/not_foggy/NEON.D01.HARV.DP1.00033/2017.csv"
with open(not_foggy_file, mode='r') as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            line_count += 1
        else:
            line_count += 1
        img_name = row["file"]
        print(img_name)
        begin_cv_img = cv2.imread(f"/content/drive/MyDrive/Capstone/data_raw/phenocamdata/{img_name}")
        begin_rgb = cv2.cvtColor(begin_cv_img, cv2.COLOR_BGR2RGB)

        # Save the image to the output folder in JPG format with the original filename
        output_path = os.path.join(output_folder, os.path.basename(img_name))
        print(f'Saving image to: {output_path}')
        cv2.imwrite(output_path, cv2.cvtColor(begin_rgb, cv2.COLOR_RGB2BGR))

    print(f'Processed {line_count} lines.')

## Creating Labels using K-mean Clustering (Not working)

In [ ]:
def read_images(directory):
    """
    This function reads and shows images from a specified directory.
    It returns a list of images.
    """
    images = []  # List to store the images

    # Iterate through each file in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            # Construct the full path to the image file
            filepath = os.path.join(directory, filename)

            # Read and normalize the image
            image = plt.imread(filepath) / 255.0

            print(f"Image shape: {image.shape}")

            # Show the image
            plt.imshow(image)
            plt.show()

            # Append the image to the list
            images.append(image)

    return images

# Specify the directory containing the images
image_directory = "/content/drive/MyDrive/Capstone/data_working/images_2017"


# Call the function to read and display images
pic = read_images(image_directory)


In [ ]:
def image_shape(directory, batch_size=10):
    """
    This function reads the images from the specific directory, appends them to an image list,
    and returns the shape of the first image in the list.
    """
    images = []  # List to store the images

    # Iterate through each file in the directory
    for filename in os.listdir(directory):
        # Check for acceptable file extensions
        if filename.endswith((".jpg", ".png")):
            # Construct the full path to the image file
            filepath = os.path.join(directory, filename)

            try:
                # Read and normalize the image
                image = plt.imread(filepath) / 255.0
                # Append the image to the list
                images.append(image)
            except Exception as e:
                print(f"Error reading {filename}: {e}")

            # Load images in batches
            if len(images) == batch_size:
                break

    if images:
        return images[0].shape
    else:
        return None  # Return None if no valid images are found

# directory containing images
image_directory = "/content/drive/MyDrive/Capstone/data_working/images_2017"

img = image_shape(image_directory, batch_size=10)
print(f"Shape of the first image: {img}")


Shape of the first image: (960, 1296, 3)


In [ ]:
# Function to load and preprocess images
def load_and_preprocess_images(image_directory):
    images = []

    # # Iterate over subdirectories (assuming they represent months)
    # for month_folder in os.listdir(image_directory):
    #     month_path = os.path.join(image_directory, month_folder)

    #     # Check if it's a directory
    #     if os.path.isdir(month_path):
    #         print(f"Loading images from {month_folder}...")

    for filename in os.listdir(image_directory):
      if filename.endswith(".jpg") or filename.endswith(".png"):
        img_path = os.path.join(image_directory, filename)
        img = cv2.imread(img_path)

        new_height = 224
        aspect_ratio = img.shape[1] / img.shape[0]
        new_width = int(new_height * aspect_ratio)
        img_resized = cv2.resize(img, (new_width, new_height))
        img_normalized = img_resized / 255.0

        images.append(img_normalized)

        #print(f"Number of images loaded from {month_folder}: {len(images)}")

    return np.array(images)


In [ ]:
import os
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import MiniBatchKMeans


In [ ]:
# Specify the main directory containing subfolders for each month
#image_directory = "/content/drive/MyDrive/Capstone/data_working/images_2017"
#January 2017
image_directory = "/content/drive/MyDrive/Capstone/data_raw/phenocamdata/NEON.D01.HARV.DP1.00033/2017/01"

# Load and preprocess images
images = load_and_preprocess_images(image_directory)

# Print debug information
print(f"Total number of images loaded: {len(images)}")

# Flatten the images
images_flattened = images.reshape(images.shape[0], -1)

# Standardize the pixel values
scaler = StandardScaler()
images_flattened_standardized = scaler.fit_transform(images_flattened)
print("Shape of images_flattened_standardized:", images_flattened_standardized.shape)

# Use Mini-Batch K-means clustering to determine labels
num_clusters = 5  # Adjust the number of clusters based on your phenological stages
batch_size = 100  # Adjust the batch size
minibatch_kmeans = MiniBatchKMeans(n_clusters=num_clusters, batch_size=batch_size, random_state=42, n_init='auto')
labels = minibatch_kmeans.fit_predict(images_flattened_standardized)
print("Labels:", labels)

Total number of images loaded: 194
Shape of images_flattened_standardized: (194, 202944)
Labels: [3 3 3 2 2 3 2 1 2 0 2 1 1 3 2 0 0 0 2 0 0 0 2 0 0 0 2 0 2 2 3 0 3 3 2 3 2
 0 0 0 2 2 3 1 1 3 2 3 3 3 3 1 1 0 0 2 0 0 0 3 0 0 0 3 0 0 2 2 0 2 2 2 2 1
 2 1 3 2 2 2 3 3 2 3 2 3 1 3 2 3 3 2 4 2 2 2 2 2 2 2 0 2 0 1 3 0 0 1 3 1 1
 3 0 3 2 4 3 3 2 3 3 2 3 3 3 3 1 3 1 1 3 4 1 1 1 3 1 1 3 4 3 2 3 2 2 3 2 3
 2 2 0 0 0 0 0 3 0 0 0 0 3 2 2 2 2 3 2 2 2 2 3 2 2 2 2 2 3 2 2 2 1 3 2 2 2
 1 3 0 0 0 0 0 0 0]


In [ ]:
# Specify the main directory containing images
#image_directory = "/content/drive/MyDrive/Capstone/data_raw/phenocamdata/NEON.D01.HARV.DP1.00033/2017/01"
image_directory = "/content/drive/MyDrive/Capstone/data_working/images_2017"
# Load and preprocess images
images = load_and_preprocess_images(image_directory)

# Print debug information
print(f"Total number of images loaded: {len(images)}")

# Flatten the images
images_flattened = images.reshape(images.shape[0], -1)

# Standardize the pixel values
scaler = StandardScaler()
images_flattened_standardized = scaler.fit_transform(images_flattened)
print("Shape of images_flattened_standardized:", images_flattened_standardized.shape)

# Use Mini-Batch K-means clustering to determine labels
num_clusters = 5  # Adjust the number of clusters based on your phenological stages
batch_size = 100  # Adjust the batch size
minibatch_kmeans = MiniBatchKMeans(n_clusters=num_clusters, batch_size=batch_size, random_state=42, n_init='auto')
cluster_labels = minibatch_kmeans.fit_predict(images_flattened_standardized)

# Use the cluster centroids as continuous labels
cluster_centroids = minibatch_kmeans.cluster_centers_

Clustering to Label dataset is not working

##Doing Simple Sequential Labeling of the non-foggy Dataset

In [5]:
import cv2
import pandas as pd
import os

# Load your CSV file with image filenames
csv_file = "/content/drive/MyDrive/Capstone/data_out/not_foggy/NEON.D01.HARV.DP1.00033/2017.csv"
df = pd.read_csv(csv_file)

# Create a new column for labels
df["Label"] = None

# Assign labels based on the order of filenames
label_counter = 1
for index, row in df.iterrows():
    # Assuming 'file' column contains the full path to the images
    filename = row['file']

    # Extract only the filename from the path
    image_name = os.path.basename(filename)

    # Assign the label
    df.at[index, 'Label'] = label_counter
    label_counter += 1

# Keep only the 'file' and 'Label' columns
df = df[['file', 'Label']]

# Specify the directory to save the Labeled CSV file
output_directory = "/content/drive/MyDrive/Capstone/data_out/not_foggy/NEON.D01.HARV.DP1.00033/"

# Save the Labeled DataFrame to a new CSV file in the specified directory
Labeled_csv_file = os.path.join(output_directory, "Labeled_data.csv")
df.to_csv(Labeled_csv_file, index=False)

##Train, Test, Validation set

In [14]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from shutil import copyfile

# Define paths
image_directory = "/content/drive/MyDrive/Capstone/data_working/images_2017"
label_csv_path = "/content/drive/MyDrive/Capstone/data_out/not_foggy/NEON.D01.HARV.DP1.00033/Labeled_data.csv"

# Load labels from CSV
labels_df = pd.read_csv(label_csv_path)

# Display the column names in your DataFrame
print("Column names in the DataFrame:", labels_df.columns)

# Split the dataset into training, testing, and validation sets
train_df, test_df = train_test_split(labels_df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

print("Done Splitting")

Column names in the DataFrame: Index(['file', 'Label'], dtype='object')
Done Splitting


In [11]:
# Define output directories for the sets
output_directory = "/content/drive/MyDrive/Capstone/data_out/not_foggy/NEON.D01.HARV.DP1.00033"
train_directory = os.path.join(output_directory, "train")
test_directory = os.path.join(output_directory, "test")
val_directory = os.path.join(output_directory, "validation")

# Create output directories if they don't exist
os.makedirs(train_directory, exist_ok=True)
os.makedirs(test_directory, exist_ok=True)
os.makedirs(val_directory, exist_ok=True)

In [15]:
# Copy images to respective directories based on the sets
def copy_images(df, source_dir, dest_dir):
    for index, row in df.iterrows():
        image_filename = row['file'].split("/")[-1]  # Extract the last part of the path
        source_path = os.path.join(source_dir, image_filename)
        dest_path = os.path.join(dest_dir, image_filename)
        copyfile(source_path, dest_path)

# Copy training set images
copy_images(train_df, image_directory, train_directory)

# Copy testing set images
copy_images(test_df, image_directory, test_directory)

# Copy validation set images
copy_images(val_df, image_directory, val_directory)

# Display the number of samples in each set
print(f"Training set size: {len(train_df)}")
print(f"Testing set size: {len(test_df)}")
print(f"Validation set size: {len(val_df)}")

Training set size: 2005
Testing set size: 558
Validation set size: 223


###Data Augmentation (if needed)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import os
import numpy as np
import matplotlib.pyplot as plt

# Set the path to your dataset
#data_path = "/content/drive/MyDrive/Capstone/data_working/images_2017"
#data_path = "/content/drive/MyDrive/Capstone/data_raw/HARV"
data_path = "/content/drive/MyDrive/Capstone/data_out/not_foggy/NEON.D01.HARV.DP1.00033/train"

# Create an ImageDataGenerator with augmentation configurations
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Output directory for augmented images
output_dir = '/content/drive/MyDrive/Capstone/data_out/augmented_HARV_train'


# Iterate through each image in the dataset and apply augmentation
for filename in os.listdir(data_path):
    img_path = os.path.join(data_path, filename)
    img = image.load_img(img_path, target_size=(224, 224))  # Assuming ResNet input size
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)

     # Generate augmented images
    i = 0
    for batch in datagen.flow(x, batch_size=1, save_to_dir=output_dir, save_prefix=filename.split('.')[0], save_format='jpeg'):
        # Display at least one augmented image
        if i == 0:
            augmented_img = image.array_to_img(batch[0])
            plt.imshow(augmented_img)
            plt.show()

        i += 1
        if i > 5:  # Adjust the number of augmented images per original image
            break

##Model Building

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data.dataset import random_split
import os

In [34]:
# Define the dataset path
train_path = "/content/drive/MyDrive/Capstone/data_out/not_foggy/NEON.D01.HARV.DP1.00033/train"
test_path = "/content/drive/MyDrive/Capstone/data_out/not_foggy/NEON.D01.HARV.DP1.00033/test"
valid_path = "/content/drive/MyDrive/Capstone/data_out/not_foggy/NEON.D01.HARV.DP1.00033/validation"
label_csv_path = "/content/drive/MyDrive/Capstone/data_out/not_foggy/NEON.D01.HARV.DP1.00033/Labeled_data.csv"
# Define the transformation for the dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [53]:
from PIL import Image
import os
import pandas as pd
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, data_path, label_csv_path, transform=None):
        self.data_path = data_path
        self.label_csv_path = label_csv_path
        self.transform = transform

        # Read the labeled CSV file
        self.df = pd.read_csv(label_csv_path)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        for index, row in self.df.iterrows():  # Fix this line
            img_filename = row['file'].split("/")[-1]
            img_path = os.path.join(self.data_path, img_filename)
            img = Image.open(img_path).convert("RGB")

            if self.transform:
                img = self.transform(img)

            # Extract the label from the CSV file
            label = self.df.at[idx, 'Label']

            return img, label


In [54]:
# Create instances of the CustomDataset with label CSV file
train_dataset = CustomDataset(train_path, label_csv_path, transform=transform)
test_dataset = CustomDataset(test_path, label_csv_path, transform=transform)
valid_dataset = CustomDataset(valid_path, label_csv_path, transform=transform)

In [55]:
# Set batch size
batch_size = 32

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# Verify the first batch to see its structure
for images, labels in train_loader:
    print("Batch Structure:")
    print(f"Images: {images.shape}, Labels: {labels}")
    break


Batch Structure:
Images: torch.Size([32, 3, 224, 224]), Labels: tensor([ 794, 1074,  254, 2638, 1175, 1340, 2662, 1641, 2484, 2599, 1302, 2629,
        2469,  677,  935,  736,  467, 1151, 1917,  494,  152, 2589, 2057,  398,
        1706, 2626, 1371, 1344, 1520, 2092, 2418,  978])


##Model building functions

In [57]:
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

# Define the convolutional functions
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

def conv1x1(in_planes, out_planes, stride=1):
    """1x1 convolution"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=1, stride=stride, bias=False)

# Define the Bottleneck block
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = conv1x1(inplanes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes, stride)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = conv1x1(planes, planes * self.expansion)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

# Define the Basic block
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

# Define the ResNet architecture
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=1000, zero_init_residual=False):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

        if zero_init_residual:
            for m in self.modules():
                if isinstance(m, Bottleneck):
                    nn.init.constant_(m.bn3.weight, 0)
                elif isinstance(m, BasicBlock):
                    nn.init.constant_(m.bn2.weight, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                conv1x1(self.inplanes, planes * block.expansion, stride),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


In [58]:
model_urls = {
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
   }


# ResNet-50
def resnet50(pretrained=False, **kwargs):
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet50']))
    return model

# ResNet-101
def resnet101(pretrained=False, **kwargs):
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet101']))
    return model

##Train Model

In [59]:
# Training function
def train_model(model, train_loader, valid_loader, num_epochs=10, learning_rate=0.001):
    criterion = nn.CrossEntropyLoss()  # Assuming you are using CrossEntropyLoss for classification
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(num_epochs):
        # Training
        model.train()
        for images, labels in train_loader:
            # Convert labels to a tensor
            labels = torch.tensor(labels, dtype=torch.long)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        with torch.no_grad():
            for images, labels in valid_loader:
                # Convert labels to a tensor
                labels = torch.tensor(labels, dtype=torch.long)

                # Forward pass for validation loss computation
                outputs = model(images)
                validation_loss = criterion(outputs, labels)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Validation Loss: {validation_loss.item()}")


In [62]:
# Usage for a regression task
num_output_nodes = 1  # Since it's a regression task
resnet50_model = resnet50(num_output_nodes)
train_model(resnet50_model, train_loader, valid_loader)

<ipython-input-59-0760623f4067>:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels, dtype=torch.long)


IndexError: ignored

In [ ]:
# Similarly, you can do the same for ResNet-101
resnet101_model = resnet101(num_output_nodes)
train_model(resnet101_model, train_loader, valid_loader)